In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [4]:
!pip install optuna

     |████████████████████████████████| 308 kB 10.2 MB/s 
     |████████████████████████████████| 81 kB 6.9 MB/s 
     |████████████████████████████████| 210 kB 52.7 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |████████████████████████████████| 113 kB 47.9 MB/s 
     |████████████████████████████████| 146 kB 57.9 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=66090ec1f6bd6919a689040d0b075241b454bdeb65e6fac7c2a921ee56e0618d
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [5]:
import optuna

In [6]:
train = pd.read_csv('/content/drive/MyDrive/project9/PCA/train_pca.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/project9/music30s_trainlabel.csv')
test =pd.read_csv('/content/drive/MyDrive/project9/PCA/test_pca.csv')


In [7]:
train_labels =train_labels.iloc[:,1:]
train_labels.head(3)


,label
0,reggae
1,reggae
2,country


In [8]:
display(train.head(3))
display(test.head(3))

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
0,-6.042859,3.360133,-0.580582,0.481972,2.052833,-1.309454,-0.927022,0.684981,0.490648,-0.438662,...,0.282041,0.624148,-0.086557,0.092189,-0.450715,-0.007269,-0.221397,0.534964,0.700290,-0.457552
1,-1.924472,-0.237534,-0.794313,-1.140835,-1.996988,-0.246758,-0.689759,2.738715,-0.320075,0.084416,...,0.578843,0.500192,-0.083044,0.277533,0.561664,1.073356,-0.761747,0.255930,0.204644,-0.195949
2,-1.809602,0.782968,1.921737,1.549557,0.894004,-0.931200,-0.953658,0.441260,-1.012534,-1.201357,...,0.782811,0.413996,-1.089124,-0.339099,-1.117547,-0.278725,-0.071495,0.067914,0.222783,0.558211


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32
0,-6.548205,-3.115204,-1.240470,-0.232519,-0.842417,-0.991010,0.969340,1.014227,1.127184,0.122532,...,0.436175,-0.539889,-0.548403,0.340316,-0.366454,0.545709,1.161120,0.065227,0.127051,0.131474
1,3.794001,-2.954257,7.572803,3.322598,-7.632030,-3.405628,-0.878950,-0.529037,1.441828,0.988991,...,0.659490,0.113366,0.995181,0.595825,-0.077484,-0.766830,1.266615,-0.225434,-0.658040,-0.819949
2,-2.089484,4.504425,-4.964538,-0.380636,-2.585731,1.882099,-0.009821,-0.850664,-0.574026,-0.611820,...,0.703774,1.404571,-0.091778,1.045946,0.452793,0.248331,0.294834,-0.466831,0.541625,0.845241


In [9]:
cols = [c for c in train.columns]

In [10]:
train = pd.concat([train,train_labels],axis=1)

In [11]:
train.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,label
0,-6.042859,3.360133,-0.580582,0.481972,2.052833,-1.309454,-0.927022,0.684981,0.490648,-0.438662,...,0.624148,-0.086557,0.092189,-0.450715,-0.007269,-0.221397,0.534964,0.700290,-0.457552,reggae
1,-1.924472,-0.237534,-0.794313,-1.140835,-1.996988,-0.246758,-0.689759,2.738715,-0.320075,0.084416,...,0.500192,-0.083044,0.277533,0.561664,1.073356,-0.761747,0.255930,0.204644,-0.195949,reggae
2,-1.809602,0.782968,1.921737,1.549557,0.894004,-0.931200,-0.953658,0.441260,-1.012534,-1.201357,...,0.413996,-1.089124,-0.339099,-1.117547,-0.278725,-0.071495,0.067914,0.222783,0.558211,country
3,0.422664,0.443629,-2.206801,2.090353,-0.798949,1.458579,-0.923127,0.087803,0.092352,-0.839420,...,-0.261078,-0.979358,-0.468351,0.587211,0.111020,-0.170387,0.178367,0.866886,-0.580060,blues
4,5.398688,-1.059610,-1.807378,1.355820,0.388511,-1.194312,0.222610,1.751299,-1.001092,-0.228982,...,-0.097138,-0.141619,-0.048795,0.314327,0.084201,-0.048339,-0.230336,0.180659,0.161392,metal


In [12]:
X = train.drop('label',axis=1)
y=train['label']

In [13]:
target_column = 'label'

In [14]:
features = [c for c in train.columns if c not in [target_column]]

In [15]:
## going by the optuna way:
import optuna
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# Which hyperparameters to tune: https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/

def objective(trial):

    # -- Tune estimator algorithm
    n_neighbors = trial.suggest_int("n_neighbors", 1, 41,2)
    #suggest_int(name, low, high[, step, log])
    weights = trial.suggest_categorical("weights", ['uniform', 'distance'])
    metric = trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’
    algorithm = trial.suggest_categorical("algorithm",['auto', 'ball_tree', 'kd_tree', 'brute'])
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric, algorithm =algorithm )
        
    # -- Cross-validate the features reduced by dimensionality reduction methods
    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
    score = cross_val_score(knn, X,y, scoring='accuracy', cv=rskfold, n_jobs = -1, error_score = 'raise')
    score = score.mean()
    return score


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, timeout = 4*60*60)

[I 2022-04-22 11:40:16,189] A new study created in memory with name: no-name-1e84fdfa-1081-4747-9e6c-7440bfcfc62b
[I 2022-04-22 11:40:18,833] Trial 0 finished with value: 0.6840740740740741 and parameters: {'n_neighbors': 13, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'brute'}. Best is trial 0 with value: 0.6840740740740741.
[I 2022-04-22 11:40:19,156] Trial 1 finished with value: 0.6792592592592592 and parameters: {'n_neighbors': 15, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'ball_tree'}. Best is trial 0 with value: 0.6840740740740741.
[I 2022-04-22 11:40:19,497] Trial 2 finished with value: 0.65 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'manhattan', 'algorithm': 'brute'}. Best is trial 0 with value: 0.6840740740740741.
[I 2022-04-22 11:40:19,994] Trial 3 finished with value: 0.6114814814814814 and parameters: {'n_neighbors': 39, 'weights': 'uniform', 'metric': 'manhattan', 'algorithm': 'ball_tree'}. Best is trial 0 with value

In [ ]:
trial = study.best_trial    
print('Accuracy: {}'.format(trial.value))   


Accuracy: 0.6999999999999998


In [ ]:
print("Best hyperparameters: {}".format(trial.params))


Best hyperparameters: {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'kd_tree'}


In [ ]:

optuna.visualization.plot_optimization_history(study)


In [16]:
knn_params = {'n_neighbors': 5, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'kd_tree'}



In [20]:

from sklearn.metrics import recall_score,precision_score,f1_score
from sklearn.metrics import classification_report
import statistics
from statistics import mean

In [18]:
%%time
N_SPLITS = 5
y_preds = []
acc= []
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits = N_SPLITS)

for fold, (train_id, test_id) in enumerate(folds.split(X,y)):
  X_train = X.iloc[train_id]
  y_train = y.iloc[train_id]
  X_valid = X.iloc[test_id]
  y_valid = y.iloc[test_id]
  model = KNeighborsClassifier(**knn_params)
  model.fit(X_train, y_train)

  valid_pred = model.predict(X_valid)
  
#print('fold:{}'.format(fold+1))
  #print('Validation Accuracy score : {}'.format(accuracy_score(y_valid, valid_pred)))
  #print('-'*100)
  acc.append(accuracy_score(y_valid, valid_pred))

  y_preds.append(model.predict(test))

CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 255 ms


In [21]:
print('the mean accuracy is: {}'.format(mean(acc)))


the mean accuracy is: 0.7044444444444444


In [22]:
from collections import Counter
def mode(data):
    """Return the most common data point from discrete or nominal data.
    ``mode`` assumes discrete data, and returns a single value. This is the
    standard treatment of the mode as commonly taught in schools:
        >>> mode([1, 1, 2, 3, 3, 3, 3, 4])
        3
    This also works with nominal (non-numeric) data:
        >>> mode(["red", "blue", "blue", "red", "green", "red", "red"])
        'red'
    If there are multiple modes with same frequency, return the first one
    encountered:
        >>> mode(['red', 'red', 'green', 'blue', 'blue'])
        'red'
    If *data* is empty, ``mode``, raises StatisticsError.
    """
    pairs = Counter(iter(data)).most_common(1)
    try:
        return pairs[0][0]
    except IndexError:
        raise StatisticsError('no mode for empty data') from None




In [23]:

y_pred = pd.DataFrame(columns=['knn_label'])
pred= []
for j in range(0,100):
    arr = [y_preds[0][j], y_preds[1][j], y_preds[2][j], y_preds[3][j],y_preds[4][j]]
    pred.append(mode(arr))

In [24]:
y_pred["knn_label"]=pred

In [25]:
y_pred

,knn_label
0,reggae
1,jazz
2,hiphop
3,reggae
4,reggae
...,...
95,rock
96,disco
97,pop
98,pop


In [26]:
y_pred.to_csv('knn_pca_pred.csv', index=False)
!cp -r '/content/knn_pca_pred.csv' /content/drive/MyDrive/project9/PCA

